# Presenting and evaluating benchmark models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os

# Evaluation scripts
from CompetitionEvaluation import load_data, structure_data, calculate_metrics
 

In [ ]:
# Where to find files
username = os.getlogin()
Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/Prediction competition 2023/Tables/'
overleafpath_figures = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/Prediction competition 2023/Figures/'
print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)

filepath = Mydropbox + 'Prediction_competition_2023/' 


## Reading in actuals

In [ ]:
df_cm_actuals = pd.read_parquet(filepath + 'cm_actuals_allyears.parquet')
df_pgm_actuals = pd.read_parquet(filepath + 'pgm_actuals_allyears.parquet')
df_cm_actuals.tail(), df_pgm_actuals.head()
# Recast to int32
df_cm_actuals['ged_sb'] = df_cm_actuals['ged_sb'].astype('int32')
df_pgm_actuals['ged_sb'] = df_pgm_actuals['ged_sb'].astype('int32')
df_pgm_actuals.index.set_names('priogrid_gid', level=1,inplace=True)
# Have to rename column name....:
df_cm_actuals.rename(columns={"ged_sb": "outcome"}, errors="raise", inplace=True)
df_pgm_actuals.rename(columns={"ged_sb": "outcome"}, errors="raise", inplace=True)
# Summarize:
print(df_cm_actuals.dtypes, df_pgm_actuals.dtypes)

In [ ]:
df_cm_actuals.describe(percentiles=[.25,.50,.75,.90,.95,.99,.992,.995])

In [ ]:
from viewser import Queryset, Column
qs = (Queryset("country_list", "country_month")

   .with_column(Column("id", from_table="country", from_column="id"))
   .with_column(Column("name", from_table="country", from_column="name"))
              
   )
countrylist = qs.publish().fetch().loc[504]


In [ ]:
countrylist.loc[69]

## Reading in benchmark prediction models: 

Two models per level:

1. cm model, based on ensemble
2. cm model, based on historical values 
3. pgm model, based on ensemble
4. pgm model, based on historical values

Ëach of these have predictions for each of four years; 2019, 2020, 2021, and 2022. The four years are collected in lists of dictionaries including dataframes and some metadata, one for each of the models above

In [ ]:
print(df_cm_actuals.query('country_id == 57'))
df_cm_actuals.loc[445:468]
df_cm_actuals.head()
df_cm_actuals.dtypes

In [ ]:
bm_cm_ensemble_poisson = []
bm_cm_ensemble_identical = []
bm_cm_constituent_poisson = []
bm_cm_actuals_bootstrap = []
bm_cm_last_historical_poisson= []

model_names = ['ensemble_poisson','ensemble_identical','constituent_poisson','bootstrap','last_historical_poisson']

bm_pgm_ensemble_poisson = []
bm_pgm_ensemble_identical = []
bm_pgm_historical_values = []
model_names = ['ensemble_poisson','ensemble_identical']

include_historical_values = True
include_constituent = True
include_pgm = True

def positive_integers(df, colname):
    df[colname] = np.round(df[colname]).astype('int32')
    df[colname][df[colname] < 0] = 0
    return(df)

colname = 'prediction'
colname = 'outcome'
for year in [2018, 2019, 2020, 2021]:
    print(year)
    first_month = (year - 1980)*12 + 1
    cm_e = {
        'year': year,
        'first_month': first_month,
        'name': 'cm_ensemble_poisson',
        'df_full': positive_integers(pd.read_parquet(filepath + 'bm_cm_ensemble_poisson_expanded_' + str(year) + '.parquet'),colname),
        'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_cm_ensemble_poisson.append(cm_e)
    
    cm_e = {
        'year': year,
        'first_month': first_month,
        'name': 'cm_ensemble_identical',
        'df_full': positive_integers(pd.read_parquet(filepath + 'bm_cm_ensemble_identical_expanded_' + str(year) + '.parquet'),colname),
        'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_cm_ensemble_identical.append(cm_e)
    
    cm_e = {
        'year': year,
        'first_month': first_month,
        'name': 'cm_constituent_poisson',
        'df_full': positive_integers(pd.read_parquet(filepath + 'bm_cm_constituent_poisson_expanded_' + str(year) + '.parquet'),colname),
        'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_cm_constituent_poisson.append(cm_e)
    
    cm_e = {
        'year': year,
        'first_month': first_month,
        'name': 'cm_actuals_bootstrap',
        'df_full': positive_integers(pd.read_parquet(filepath + 'bm_cm_bootstrap_expanded_' + str(year) + '.parquet'),colname),
        'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_cm_actuals_bootstrap.append(cm_e)
    
    if include_historical_values:
        cm_hv = {
            'year': year,
            'first_month': first_month,
            'name': 'cm_historical_poisson',
            'df_full': positive_integers(pd.read_parquet(filepath + 'bm_cm_last_historical_poisson_expanded_' + str(year) + '.parquet'),colname),
            'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
        }
        bm_cm_last_historical_poisson.append(cm_hv)
        
    if include_pgm:
        pgm_e = {
            'year': year,
            'first_month': first_month,
            'name': 'pgm_ensemble_poisson',
            'df_full': positive_integers(pd.read_parquet(filepath + 'bm_pgm_ensemble_poisson_expanded_' + str(year) + '.parquet'),colname),
            'actuals': df_pgm_actuals.loc[first_month: first_month + 12 - 1],
        }
        pgm_e['df_full'].index.set_names('priogrid_gid', level=1,inplace=True)
        bm_pgm_ensemble_poisson.append(pgm_e)
        
        pgm_e = {
            'year': year,
            'first_month': first_month,
            'name': 'pgm_ensemble_identical',
            'df_full': positive_integers(pd.read_parquet(filepath + 'bm_pgm_ensemble_identical_expanded_' + str(year) + '.parquet'),colname),
            'actuals': df_pgm_actuals.loc[first_month: first_month + 12 - 1],
        }
        pgm_e['df_full'].index.set_names('priogrid_gid', level=1,inplace=True)
        bm_pgm_ensemble_identical.append(pgm_e)
        
    if False:
        pgm_hv = {
            'year': year,
            'first_month': first_month,
            'name': 'pgm_historical_values',
            'df_full': positive_integers(pd.read_parquet(filepath + 'bm_pgm_historical_values_' + str(year) + '.parquet'),colname),

            'actuals': df_pgm_actuals.loc[first_month: first_month + 12 - 1],
        }
        bm_pgm_historical_values.append(pgm_hv)

In [ ]:
bm_cm_ensemble_poisson[3]['df_full'].loc[494].loc[67].describe()

In [ ]:
# Restructuring, evaluating:

# Evaluation parameters:
ign_bins = [0, 0.5, 2.5, 5.5, 10.5, 25.5, 50.5, 100.5, 250.5, 500.5, 1000.5]
#ign_bins = [0, 0.5, 1000]
#bm_cm_constituent_poisson,
bm_list = [bm_pgm_ensemble_poisson, bm_pgm_ensemble_poisson, bm_cm_ensemble_poisson,bm_cm_ensemble_identical,bm_cm_last_historical_poisson,bm_cm_actuals_bootstrap]
#bm_list = [bm_cm_ensemble_poisson,bm_cm_ensemble_identical,bm_cm_last_historical_poisson,bm_cm_actuals_bootstrap]

# this is an absolute horror - need to make sure that spatial units correspond to bm_list
spatial_units = ['priogrid_gid','priogrid_gid','country_id','country_id','country_id','country_id']

for model_list,spatial_unit in zip(bm_list,spatial_units):
    for item in model_list:
        print(item['name'], item['year'])
#        print(item['df_full'].query('draw == 0').describe())
        item['observed'], item['predictions'] = structure_data(item['actuals'], item['df_full']) # structure data as xarrays that the xskillscore.crps_ensemble wants
        item['crps'] = calculate_metrics(item['observed'], item['predictions'], metric = 'crps', aggregate_over=['month_id', spatial_unit]) # calculates crps.
        item['crps_by_month'] = calculate_metrics(item['observed'], item['predictions'], metric = 'crps', aggregate_over=spatial_unit) # calculates crps.
        item['crps_by_country'] = calculate_metrics(item['observed'], item['predictions'], metric = 'crps', aggregate_over='month_id') # calculates crps.
        item['ign'] = calculate_metrics(item['observed'], item['predictions'], metric = "ign", bins = ign_bins, aggregate_over=['month_id', spatial_unit])
        item['ign_by_month'] = calculate_metrics(item['observed'], item['predictions'], metric = "ign", bins = ign_bins, aggregate_over=spatial_unit)
        item['ign_by_country'] = calculate_metrics(item['observed'], item['predictions'], metric = "ign", bins = ign_bins, aggregate_over='month_id')
        item['mis'] = calculate_metrics(item['observed'], item['predictions'], metric = "mis", aggregate_over=['month_id', spatial_unit])
        item['mis_by_month'] = calculate_metrics(item['observed'], item['predictions'], metric = "mis", aggregate_over=spatial_unit)
        item['mis_by_country'] = calculate_metrics(item['observed'], item['predictions'], metric = "mis", aggregate_over='month_id')


        

In [ ]:
countrylist.loc[246]

In [ ]:

columns_to_plot = [
    ('prediction',    '1%'),
    ('prediction',    '5%'),
    ('prediction',   '10%'),
    ('prediction',   '20%'),
    ('prediction',   '50%'),
    ('prediction',   '80%'),
    ('prediction',   '90%'),
    ('prediction',   '95%'),
    ('prediction',   '99%'),
]
ctp1 = [
    ('prediction',  'mean'),]
ctp2 = [(   'actuals',  'mean')] # Separate set for separate color/pattern


for model_list in bm_list:
    for item in model_list:
        df_actuals = item['actuals'].reorder_levels(['country_id','month_id'])
        df_actuals.rename(columns={'outcome':'actuals'},inplace=True)
        df_full = item['df_full'].reorder_levels(['country_id','month_id','draw'])
        df_full.rename(columns={'outcome':'prediction'},inplace=True)
        df_merged = pd.merge(df_actuals, df_full,left_index=True,right_index=True)
        for c in [(57,'Ethiopia'),(67,'Algeria'),(69,'Cameroon'),(131,'Saudi Arabia'),(246,'South Sudan')]:
            title = 'Model ' + item['name'] + ', ' + c[1] + ' ' + str(item['year'])
            fig, axs = plt.subplots(figsize=(16, 4))
            df_description = df_merged.loc[c[0]].groupby('month_id').describe(percentiles=[.01,.05,.1,.2,.5,.8,.9,.95,.99])
            df_description[columns_to_plot].plot(use_index=True,ylabel='Battle-related deaths',ax=axs,title=title,colormap='coolwarm')
            df_description[ctp1].plot(ax=axs,color='gray',  linestyle='dashed', linewidth=2)
            df_description[ctp2].plot(ax=axs,color='black', linewidth=3)
            axs.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size':8})
            fig.savefig(overleafpath_figures+'bm_predictions/'+'predictions_and_actuals_' + c[1] + '_' + str(item['year']) + '_' + item['name'])
    

In [ ]:
df_description.columns

In [ ]:
columns_to_plot = [
    'actuals',
    ('outcome',   '5%'),
    ('outcome',   '10%'),
    ('outcome',   '20%'),
    ('outcome',   '50%'),
    ('outcome',   '80%'),
    ('outcome',   '90%'),
    ('outcome',   '95%')]


for c in [(57,'Ethiopia'),(69,'xx'),(130,'yy'),(246,'zz')]:
    
    df_country = item['df_full'].query(f'country_id=={c[0]}')
    df_actuals = item['actuals'].query(f'country_id=={c[0]}')
    df_actuals['actuals'] = df_actuals['outcome']
    df_description = df_country.groupby('month_id').describe(percentiles=[.05,.1,.2,.5,.8,.9,.95])
#    df_description.reset_index(inplace=True)
    df_joined = pd.concat([df_actuals,df_description], ignore_index=True)

    df_joined[columns_to_plot].plot()

In [ ]:
item['df_full']

In [ ]:
df_joined = pd.concat([df_actuals,df_description], ignore_index=True)
df_joined.head()

In [ ]:
df_actuals.head()

In [ ]:
df_actuals.describe(), df_description.describe()

In [ ]:
print(df_description.head())
x = df_description['month_id']
y=df_description[('outcome',   '10%')]
z=df_description[('outcome',   '90%')]
plt.plot(x,y,z)


In [ ]:
np.percentile?

# Assembling evaluation tables


In [ ]:
# Create table A for each model with:
# one row for each year plus one for mean over years
# one col for crps 
# one col for ign 
# one col for mis
# Create table B with the same, but metrics per month

# Table A:
for model_list in bm_list:
    year_list = []
    for item in model_list:
        metrics = pd.concat([item['crps'],item['ign'],item['mis']],axis=1)
        metrics['year'] = item['year']
        year_list.append(metrics)
    table_annual = pd.concat(year_list,axis=0)   
    table_annual.set_index(str('year'),inplace=True)
    table_annual.loc['Mean'] = table_annual.mean()
    table_filename = overleafpath + 'bm_evaluation/' + item['name'] + '_aggregated' + '.tex'
    with open(table_filename, 'w') as tf:
        tf.write(table_annual.to_latex(float_format="{:.2f}".format))

# Table B:
for model_list in bm_list:
    year_list = []
    for item in model_list:
        metrics = pd.concat([item['crps_by_month'],item['ign_by_month'],item['mis_by_month']],axis=1)
        metrics['month'] = metrics.index - (item['year']-1980)*12
        metrics['month'] = metrics['month'].astype(int)
        metrics['year'] = item['year']
        year_list.append(metrics)
    table_monthly = pd.concat(year_list,axis=0)   
    table_monthly.set_index(['year','month'],inplace=True)
    table_monthly_aggregated = table_monthly.groupby('month').agg('mean')
    #table.loc['Mean'] = table.mean()
    table_filename = overleafpath + 'bm_evaluation/' + item['name'] + '_monthly' + '.tex'
    with open(table_filename, 'w') as tf:
        tf.write(table_monthly_aggregated.to_latex(float_format="{:.2f}".format))

In [ ]:
table_annual.to_latex(float_format="{:.2f}".format,column_format='lrrr')

In [ ]:
metrics['month'] = metrics.index - (2021-1980)*12
metrics

In [ ]:
# Checking whether balanced panel:

if include_historical_values:
    print(bm_cm_historical_values[0]['df_full'].head())
    print('Number of missing?',bm_cm_historical_values[0]['df_full'].isnull().sum())
    print(bm_cm_historical_values[0]['df_full'].describe())
    for m in range(445,457):
        df = bm_cm_historical_values[0]['df_full'].loc[m]
        print(m,len(df))
        df2 =df.groupby(["country_id"]).agg({'prediction': ["count"]})
        print(df2.describe())
    print(990*2292)

In [ ]:

if include_historical_values:
    print(bm_cm_historical_values[0]['df_full'].describe())
    print(bm_cm_historical_values[0]['df_full'].query('draw == 0').describe())
    print(bm_cm_historical_values[0]['df_full'].head())

In [ ]:
bm_pgm_ensemble

In [ ]:
print(len(item['df_full']))
print(len(item['df_full'])/12)
print(len(item['df_full'])/(12*990))

In [ ]:
from IgnoranceScore import ensemble_ignorance_score, _ensemble_ignorance_score
import numpy as np
observations = [0, 1, 50, 500]
forecasts = np.array([[0, 0, 0, 0, 0],
                      [1, 1, 1, 2, 55],
                      [500, 49, 52, 52, 500],
                      [49, 49, 49, 49, 500]])
bins = [0, 0.5, 10.5, 50.5, 100.5, 1000.5]
res = ensemble_ignorance_score(observations, forecasts, prob_type=3, ign_max=None, round_values=False, axis=-1, bins = bins, low_bin=0, high_bin=1000)
res

In [ ]:
from CompetitionEvaluation import load_data, structure_data, calculate_metrics
 
observed, predictions = load_data(forecasts_path=filepath + "cm_benchmark_ensemble_550.parquet",
                                    observed_path=filepath + "cm_actuals.parquet")
predictions["prediction"] = predictions["prediction"].replace(-1, 0)
observed, predictions = structure_data(observed, predictions)
metrics = calculate_metrics(observed, predictions, metric = "ign", round_values = True)
metrics

In [ ]:
help(calculate_metrics)

In [ ]:
        

#observed, predictions = load_data(args.o, args.p) # read parquet files to pandas
observed, predictions = structure_data(df_pgm_actuals, df_bm_pgm_historical_values) # structure data as xarrays that the xskillscore.crps_ensemble wants
metrics = calculate_metrics(observed, predictions) # calculates crps.

In [ ]:
df_bm_cm_ensemble = pd.read_parquet(filepath + 'cm_benchmark_ensemble_550.parquet')
df_bm_cm_ensemble.describe()

In [ ]:
df_bm_cm_ensemble.head()

In [ ]:
df_bm_pgm_historical_values = pd.read_parquet(filepath + 'pgm_benchmark_historical_values_step_3.parquet')
df_bm_pgm_historical_values.describe()

In [ ]:
#observed, predictions = load_data(args.o, args.p) # read parquet files to pandas
observed, predictions = structure_data(df_cm_actuals, df_bm_cm_ensemble) # structure data as xarrays that the xskillscore.crps_ensemble wants
metrics = calculate_metrics(observed, predictions) # calculates crps.

In [ ]:
metrics

In [ ]:
# Read in for all 12 steps
from datetime import datetime
print("Cell started to run:", datetime.now())

df_pgm_hv = []
for step in range(3,14+1):
    df = pd.read_parquet(filepath + 'pgm_benchmark_historical_values_step_' + str(step) + '.parquet')
    print(step, df.describe())
    df_pgm_hv.append(df)
    
print("Cell run ended:", datetime.now())

In [ ]:
print("Cell started to run:", datetime.now())
i = 3
for df in df_pgm_hv:
    print('step',i,datetime.now())
    observed, predictions = structure_data(df_pgm_actuals, df) # structure data as xarrays that the xskillscore.crps_ensemble wants
    metrics = calculate_metrics(observed, predictions) # calculates crps.
    print(metrics)
    i=i+1
print("Cell run ended:", datetime.now())



# Read in the sc-type prediction files


In [ ]:
df_bm_pgm_ensemble2022 = pd.read_parquet(filepath + 'bm_pgm_ensemble_2022.parquet')
df_pgm_actuals_2022 = df_pgm_actuals.loc[505:516]
df_bm_pgm_ensemble2022.tail()

In [ ]:

observed, predictions = structure_data(df_pgm_actuals_2022, df_bm_pgm_ensemble2022) # structure data as xarrays that the xskillscore.crps_ensemble wants
metrics = calculate_metrics(observed, predictions) # calculates crps.
metrics

# Creating samples based on point predictions

Assuming Poisson distributions

In [ ]:
cm_ensemble_aggregated = pd.read_parquet(filepath + 'cm_benchmark_ensemble_550_aggregated.parquet')

print(cm_ensemble_aggregated.describe())
print(cm_ensemble_aggregated.head())

In [ ]:
# Strip down to a year of sc predictions:
df_cm_ensemble = []
for step in range(3,14+1):
    df = cm_ensemble_aggregated['mean_log_prediction'].loc[442+step]
    df = pd.DataFrame(df[df.index.get_level_values('step').isin([step])])
    df['prediction'] = np.expm1(df['mean_log_prediction'])
    df_cm_ensemble.append(df)

df_cm_ensemble_stripped = pd.concat(df_cm_ensemble)
print(df_cm_ensemble_stripped.describe())
print(df_cm_ensemble_stripped.tail(40))


In [ ]:
test = np.